# Tarea 2

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes

### Instrucciones generales

- Grupos de máximo 4 personas.
- Esta prohibido compartir las respuestas con otros grupos.
- Indicios de copia serán penalizados con la nota mínima.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente, si utiliza material extra debe citarlo.


### Integrantes

## MARCO A. BUSTAMANTE C.

## Contexto

El discurso de odio es cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo de personas en una relación asimétrica de poder, tal como la raza, la etnia, el género, la orientación sexual, la religión, la nacionalidad, una discapacidad u otra característica similar.

En cambio, la incivilidad se refiere a cualquier comportamiento o actitud que rompe las normas de respeto, cortesía y consideración en la interacción entre personas. Esta puede manifestarse de diversas formas, tal como insultos, ataques personales, sarcasmo, desprecio, entre otras.

En esta tarea tendrán a su disposición un dataset de textos con las etiquetas `odio`, `incivilidad` o `normal`. La mayor parte de los datos se encuentra en español de Chile. Con estos datos, deberán entrenar un modelo que sea capaz de predecir la etiqueta de un texto dado.

El corpus para esta tarea se compone de 3 datasets:  
- [Multilingual Resources for Offensive Language Detection de Arango et al. (2022)](https://aclanthology.org/2022.woah-1.pdf#page=136)
- [Dataton UTFSM No To Hate (2022)](http://dataton.inf.utfsm.cl/)
- Datos generados usando la [API de GPT3 (modelo DaVinci 03)](https://platform.openai.com/docs/models/gpt-3).

Agradecimientos a los autores por compartir los datos y a David Miranda, Fabián Diaz, Santiago Maass y Jorge Ortiz por revisar y reetiquetar los datos en el contexto del curso "Taller de Desarrollo de Proyectos de IA" (CC6409), Departamento de Ciencias de la Computación, Universidad de Chile.

Los datos solo pueden ser usados con fines de investigación y docencia. Está prohibida la difusión externa.


## Tarea a resolver

Para esta tarea 2, buscaremos desarrollar un *benchmark* sobre una tarea de clasificación de NLP. Un benchmark es básicamente utilizar diferentes técnicas para resolver una misma tarea específica, en este caso seguiremos buscando alternativas para resolver el problema de clasificación de la tarea 1. Particularmente, se le pide:

- Implementar una arquitectura en RNN utilizando PyTorch.
- Utilizar transformers para revolver el problema de clasificación, en especifico utilizar BETO.
- Utilizar algún LLM utilizando Zero y Few short learning para resolver el problema de clasificación.


### Cargar el dataset


En esta sección, cargaremos el dataset desde el repositorio del módulo. Para ello ejecute las siguientes líneas:

In [1]:
import pandas as pd

In [2]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")


### Analizar los datos

En esta sección analizaremos el balance de los datos. Para ello se imprime la cantidad de tweets de cada dataset agrupados por la intensidad de sentimiento.

In [3]:
dataset_df.sample(5)

,id,texto,clase
8540,9305,"@user siguiendo esa lógica, los cuicos no serí...",incivilidad
11993,10331,@user NO fueron a votar esos IGNORADOS e ignor...,odio
2339,6095,@user En Perú tmb son wns.. como tanto voto ha...,incivilidad
8426,12963,"@user Das pena, eres agresiva discriminadora, ...",odio
3429,11675,@user @user Una vergüenza que los extranjeros ...,odio


In [4]:
dataset_df["clase"].value_counts()

clase
incivilidad    5424
normal         4280
odio           2510
Name: count, dtype: int64

In [5]:
target_classes = list(dataset_df['clase'].unique())
target_classes

['normal', 'incivilidad', 'odio']

### Instalar librerias

Puede usar esta celda para instalar las librerías que estime necesario.

In [ ]:
%%capture



### Importar librerías

En esta sección, importamos la liberías necesarias para el correcto desarrollo de esta tarea. Puede utilizar otras librerías que no se en encuentran aquí, pero debe citar su fuente.

In [ ]:
!pip uninstall torch torchvision torchaudio

!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cpu
!pip uninstall scipy
!pip install scipy==1.11.4
!pip install torchtext
!pip install scikit-plot

Found existing installation: torch 2.3.0+cpu
Uninstalling torch-2.3.0+cpu:
  Would remove:
    /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/bin/convert-caffe2-to-onnx
    /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/bin/convert-onnx-to-caffe2
    /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/bin/torchrun
    /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/functorch/*
    /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/torch-2.3.0+cpu.dist-info/*
    /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/torch/*
    /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/torchgen/*
Proceed (Y/n)? ERROR: O

In [6]:
import nltk
import numpy as np

from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

# importe aquí sus clasificadores

import matplotlib.pyplot as plt

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

# Pytorch imports
import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

from torch import nn
from torch.nn import functional as F

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

from torch.optim import Adam


/home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_

### Crear un clasificador basado en RNN

En esta parte de le pide definir un clasificador utilizando `PyTorch` con alguna arquitectura en Redes Recurrentes. Para ello debe realizar todos los pasos vistos en el tutorial 5, por lo que se recomienda revisarlo. Importante, no puede replicar ningún ejemplo de los del tutorial 5, debe proponer sus propias arquitecturas. Se le recomienda leer como utilizar variaciones de la RNN, como la LSTM o GRU en `Pytorch`.

Para completa esta parte deberá replicar el flujo de trabajo de como utilizar `PyTorch`. Esta esctrictamente prohibido utilizar variaciones que resuelvan directamente este problema, como `PyTorch Lightning` o `TensorFlow`. Los pasos a completar son los siguientes:

#### Cargar el dataset.

#### Definir el vocabulario.

#### Cargar el `DataLoader`

Recuerde que podría necesitar una función intermedia para procesar cada batch durante el entrenamiento, pero no es obligatorio hacerlo.

In [ ]:
t



#### Definir la red recurrente.

Recuerde que debe difirnir los hyperparametros que estime conveniente.

In [102]:
embed_len = 50
hidden_dim = 50
n_layers=1


class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_len, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_len)
        self.rnn = nn.GRU(embed_len, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, X, lengths=None):
        emb = self.embedding(X)  # (batch, seq_len, embed_len)
        
        if lengths is not None:
            packed = torch.nn.utils.rnn.pack_padded_sequence(
                emb, lengths.cpu(), batch_first=True, enforce_sorted=False
            )
            _, hidden = self.rnn(packed)
        else:
            _, hidden = self.rnn(emb)

        return self.linear(hidden[-1])  # (batch, num_classes)



#### Funciones de entrenamiento y evaluación.

Para esta parte, puede utilizar las funciones vista en el tutorial directamente. Pero es su reponsabilidad ajustarlas a su código.

##### Entrenamiento del modelo

Ejecute el entrenamiento de su modelo propuesto.

In [106]:
embed_len = 50
hidden_dim1 = 50
hidden_dim2 = 60
hidden_dim3 = 75
n_layers=1
learning_rate = 0.001
epochs = 10

train_dataset = load_df(train_df)
test_dataset = load_df(test_df)

train_dataset = to_map_style_dataset(train_dataset)
test_dataset = to_map_style_dataset(test_dataset)



from torch.utils.data import DataLoader

batch_size = 32  # o el que uses

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=vectorize_batch,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=vectorize_batch,
)
rnn_classifier = RNNClassifier(len(vocab), embed_len, hidden_dim1, len(target_classes))
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate) 


#train_loader = DataLoader(train_dataset, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
# test_loader  = DataLoader(test_dataset , batch_size=1024, collate_fn=vectorize_batch)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)


100%|██████████| 306/306 [00:05<00:00, 53.54it/s]


In [104]:
def MakePredictions(model, loader, device="cpu"):
    model.eval()
    model.to(device)
    Y_true, Y_pred = [], []
    
    with torch.no_grad():
        for X, Y, lengths in loader:
            X, Y, lengths = X.to(device), Y.to(device), lengths.to(device)
            logits = model(X, lengths)
            preds = torch.argmax(logits, dim=1)
            Y_true.append(Y.cpu())
            Y_pred.append(preds.cpu())

    Y_true = torch.cat(Y_true)
    Y_pred = torch.cat(Y_pred)

    return Y_true, Y_pred

##### Evaluacion del modelo

Ejecute la evaluación de su modelo, y genere un reporte de evaluación similar al de la pregunta anterior.

In [105]:
Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)


In [86]:
print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

Test Accuracy : 0.3499795333606222

Classification Report : 
              precision    recall  f1-score   support

      normal       0.35      1.00      0.52       856
 incivilidad       0.00      0.00      0.00      1085
        odio       0.23      0.01      0.01       502

    accuracy                           0.35      2443
   macro avg       0.19      0.33      0.18      2443
weighted avg       0.17      0.35      0.18      2443


Confusion Matrix : 
[[ 852    0    4]
 [1079    0    6]
 [ 499    0    3]]


/home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defin

Finalmente, análice sus resultados, ¿Porqué cree que obtuvo esos resultados?, ¿Es mejor que sólo utilizar Word Embeddings, porque?. Justique.

### Transformers BERT.

Para esta tarea se le piden crear una representación de texto usando la arquitectura basada en transformers, BERT:



In [ ]:
!pip install transformers


#### Import BETO

Para esto debe importar el tokenizador y el modelo BETO.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

#### Ejemplo de extracción de features.

Luego, debe cargar los modelos pre-entrenados:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = AutoModelForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', output_hidden_states=True)

Una vez cargado BETO, debe utilizarlo para extraer los embeddings asociados a la texto de su corpus. Se espera que usted realice lo siguiente:



Tokenizamos el texto para extraer los ids a cada palabra en el vocabulario interno de BETO, se recomienda utilizar el padding, trunctation, y max_length para considerar oraciones de diferentes tamaños.

Luego, debe verificar si cada uno de los ids extraídos se encuentran en la misma máquina donde fue cargado el modelo (CPU o GPU), se recomienda dejar todo en GPU.

In [ ]:
# oración
sentence = "Hola, que tal? me gusta mucho el curso de NLP"

# extraemos los ids de los tokens, se recomienda definir los valores de las variables:
#  padding, truncation, max_length debido a que las secuencia de texto puede tener diferentes largos
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=512)
# revisamos si cada de los ids, se encuentran en la misma máquina que el modelo (GPU o CPU)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
inputs

Una vez, extraídos los ids, usted debe obtener los estados ocultos de las ultimas capas de BETO.

In [ ]:
# Extraemos la features
outputs = model(**inputs)

# ahora `outputs` tendrá el atributo `hidden_states`
hidden_states = outputs.hidden_states


Finalmente, debe guardar los embeddings en CPU los embeddings del token [CLS] que será usados en la clasificación del análisis de sentimientos.

In [ ]:
with torch.no_grad():
# Seleccionamos la última capa y obtenemos el primer token ([CLS]) para cada ejemplo
# Estos son los embeddings que normalmente se usan para tareas de clasificación.
  cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()


#### Extraer features de todo el dataset

Considerando lo anterior, usted debe implementar la función `get_beto_features_in_batches` para extraer los features de BETO (los estados ocultos y los embeddings del token [CLS]).

Esta función recibe dos parámetros, el texto a vectorizar y un tamaño de batch, debido a que es extramadamente recomendable a que procesen el texto por lotes, ya que si cargan todos el modelo se les agotará la memoria RAM.

In [ ]:
# Función para procesar los textos en lotes y obtener las características de BETO

def get_beto_features_in_batches(texts, batch_size):

  pass

Ahora extraíga los features, un punto importante es que la extracción de features podría tomar alrededor de una a dos horas dependiendo del tamaño del batch que utilicen.

In [ ]:
train_embs = get_beto_features_in_batches(..., ...)

#### Clasificación

Una vez extraído los features de BETO, realice la clasificación de los embeddings obtenidos. Debe implementar dos clasificadores a su elección.

#### Reporte de evaluación

Una vez realizada la clasificación, realice el reporte de clasificación y el análsis de la matriz confusión para ambos clasificadores.

Recuerde que para hacer esto, debe extraer los features del dataset de testing.

Finalmente, que puede decir de los resultados obtenidos. ¿Se diferencia de los resultados obtenidos de la red RNN? ¿A que se debe esto? Justifique

### Large Language Model

##### Zero Shot Learning

Utilizando la técnica de zero shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.

##### Few Shot Learning

Utilizando la técnica de few shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.

In [ ]:
 Asegurar que la conversión fue exitosa
print("Tipo de train_dataset:", type(train_dataset))
print("Primeros elementos:", list(train_dataset)[:3])  # Ver primeros 3 elementos